In [1]:
# Imports.
import pandas as pd
import csv
import re
from tqdm import tqdm
import ast
from multiprocessing import Pool, cpu_count, mut

In [2]:
# Get datasets.
grams = pd.read_csv("../data/grams.csv")
print("Length of USDA portions dataset:", len(grams))
print("")

ingr = pd.read_csv("../data/recipes_candidate_freq10.csv")
print("Lenght of recipes dataset:", len(ingr))
print("")

link = pd.read_csv("../data/linked_df.csv")
print("Lenght of link dataset:", len(link))
print("")

Length of USDA portions dataset: 32614

Lenght of recipes dataset: 14312

Lenght of link dataset: 2573



In [3]:
grams

,Food_Code,Main_Food_Description,Subcode_Description,WWEIA_Category_Description,Portion_Description,Portion_Weight_(g)
0,11000000,"milk, human",NaN,human milk,cup,246.0
1,11000000,"milk, human",NaN,human milk,quantity not specified,0.0
2,11000000,"milk, human",NaN,human milk,fl oz,30.8
3,11100000,"milk, nfs",NaN,"milk, reduced fat",cup,244.0
4,11100000,"milk, nfs",NaN,"milk, reduced fat",fl oz,30.5
...,...,...,...,...,...,...
32609,95330500,"fluid replacement, 5% glucose in water",NaN,sport and energy drinks,quantity not specified,120.0
32610,95342000,"fruit juice, acai blend",NaN,other fruit juice,quantity not specified,248.0
32611,95342000,"fruit juice, acai blend",NaN,other fruit juice,fl oz (no ice),31.0
32612,95342000,"fruit juice, acai blend",NaN,other fruit juice,fl oz (with ice),23.0


In [4]:
link

,Ingr,Match,Score
0,lemon pepper,pepper steak,0.500000
1,haas avocados,NaN,0.000000
2,chicken stock,"chicken, chicken roll, roasted",0.577350
3,mccormick® nutmeg,NaN,0.000000
4,san marzano plum tomatoes,"plum, raw",0.353553
...,...,...,...
2568,chipotle in adobo puree,"chipotle dip, regular",0.288675
2569,apple cider vinegar,apple cider,0.816497
2570,carrot and celery sticks,"muffin, carrot",0.353553
2571,gorgonzola,NaN,0.000000


In [5]:
ingr

,title,food,quantity,unit
0,Infineon Raceway Baked Beans,"['skirt steak', 'pepper', 'bacon', 'onion', 'j...","['2', None, 'about 1', '1 1/2', '2', '3', '1',...","['pounds', None, 'cup', 'cups', 'cup', 'tables..."
1,Saffron Jewel Rice,"['saffron threads', 'sugar', 'kosher salt', 'o...","['1/2', '1/3', None, '2', '1', '4', '1', '1', ...","['teaspoon', 'cup', None, 'strips', None, 'tab..."
2,Beet Salad,"['beets', 'wine vinegar', 'dijon mustard', 'ca...","['1 1/2', '3', '1', '1', '1/3', None, '6', '3'...","['pounds', 'tablespoons', 'teaspoon', 'teaspoo..."
3,Happy Holly's Banana Cream Pie,"['all-purpose flour', 'sugar', 'kosher salt', ...","['1 1/2', '2', '1/2', '6', '3/4', '1/3', '1/4'...","['cups', 'teaspoons', 'teaspoon', 'tablespoons..."
4,Zucchini Carpaccio,"['zucchini', 'pepper', 'olive oil', 'lemon', '...","['2', None, '1', '1/2', '1', '1/4', None]","[None, None, 'tablespoon', None, None, 'piece'..."
...,...,...,...,...
14307,Quick Spicy Pickled Beets,[],[],[]
14308,Eggnog-Chocolate Cake Roll,"['vegetable oil', 'all-purpose flour', 'sugar'...","['1/2', '1 1/4', '2', '1/2', '1/4', '1/2', '1/...","['cup', 'cups', 'tablespoons', 'cup', 'cup', '..."
14309,Apple Chimichangas,"['butter', 'sugar', 'cinnamon', 'apple', 'pear...","['1/4', '1/3', '1', '1', '1', '1/4', '4', '1',...","['cup', 'cup', 'teaspoon', None, None, 'cup', ..."
14310,Summer Corn Salad,"['corn', 'belgian endive', 'olive oil', 'peppe...","['4', '2', '2', None, '3', '1/2', '1/4', '1/2'...","['ears', 'heads', 'tablespoons', None, 'bunch'..."


In [6]:
ingr.food = ingr.food.apply(ast.literal_eval)
ingr.quantity = ingr.quantity.apply(ast.literal_eval)
ingr.unit = ingr.unit.apply(ast.literal_eval)

In [7]:
ingr.quantity[0]

['2',
 None,
 'about 1',
 '1 1/2',
 '2',
 '3',
 '1',
 '2',
 None,
 '15',
 '15',
 '15',
 '1/3',
 '2',
 '3',
 '1/3',
 None]

In [8]:
# Convert all instnaces of pound to oz.
_UNITS_TO_CONVERT = ["pound", "pounds", "lb", "lbs.", "lbs"]
def quantity_adjuster(x):
    for i in range(len(x.unit)):
        if x.unit[i] in _UNITS_TO_CONVERT:
            x.unit[i] = "oz"
            if type(x.quantity[i]) is list and len(x.quantity[i]) > 0:
                x.quantity[i] = float(x.quantity[i]) * 16
        else:
            try:
                x.quantity[i] = float(x.quantity[i])
            except:
                pass
    return x

In [9]:
ingr = ingr.apply(quantity_adjuster, axis=1)

In [10]:
ingr

,title,food,quantity,unit
0,Infineon Raceway Baked Beans,"[skirt steak, pepper, bacon, onion, jalapenos,...","[2, None, about 1, 1 1/2, 2.0, 3.0, 1.0, 2.0, ...","[oz, None, cup, cups, cup, tablespoons, teaspo..."
1,Saffron Jewel Rice,"[saffron threads, sugar, kosher salt, orange z...","[1/2, 1/3, None, 2.0, 1.0, 4.0, 1.0, 1.0, 3.0,...","[teaspoon, cup, None, strips, None, tablespoon..."
2,Beet Salad,"[beets, wine vinegar, dijon mustard, caraway s...","[1 1/2, 3.0, 1.0, 1.0, 1/3, None, 6.0, 3.0, No...","[oz, tablespoons, teaspoon, teaspoon, cup, Non..."
3,Happy Holly's Banana Cream Pie,"[all-purpose flour, sugar, kosher salt, butter...","[1 1/2, 2.0, 1/2, 6.0, 3/4, 1/3, 1/4, 2.0, 3.0...","[cups, teaspoons, teaspoon, tablespoons, cup, ..."
4,Zucchini Carpaccio,"[zucchini, pepper, olive oil, lemon, leek, par...","[2.0, None, 1.0, 1/2, 1.0, 1/4, None]","[None, None, tablespoon, None, None, piece, None]"
...,...,...,...,...
14307,Quick Spicy Pickled Beets,[],[],[]
14308,Eggnog-Chocolate Cake Roll,"[vegetable oil, all-purpose flour, sugar, malt...","[1/2, 1 1/4, 2.0, 1/2, 1/4, 1/2, 1/2, 1/4, 1/4...","[cup, cups, tablespoons, cup, cup, teaspoon, t..."
14309,Apple Chimichangas,"[butter, sugar, cinnamon, apple, pear, tequila...","[1/4, 1/3, 1.0, 1.0, 1.0, 1/4, 4.0, 1.0, None]","[cup, cup, teaspoon, None, None, cup, None, cu..."
14310,Summer Corn Salad,"[corn, belgian endive, olive oil, pepper, cila...","[4.0, 2.0, 2.0, None, 3.0, 1/2, 1/4, 1/2, 1/2,...","[ears, heads, tablespoons, None, bunch, None, ..."


In [21]:
# Get the weights of the ingredients and match ingr that has poor score to a usda ingr.

all_matched = link.Ingr.to_list()



def get_weights(x):
    _, x = x
    _total_matched = 0
    _total_processed = 0
    res_dict = x.to_dict()
    # Create list for portion_weights. (Default value is None)
    res_dict["portion_weight"] = [None for _ in range(len(x.food))]
    for i in range(len(x.food)):
        if x.food[i] not in all_matched:
            continue
        matched = ""
        score = link.loc[link.Ingr == x.food[i]].Score.values[0] > .85
        unit = x.unit[i]
        if score >= 0.85:
            matched = link.loc[link.Ingr == x.food[i]].Match.values[0]
        else:
            # Special rules for certain ingredients.
            if "chicken" in x.food[i]:
                    matched = "chicken breast, baked, broiled, or roasted, skin eaten, from raw"
            elif "fish" in x.food[i]:
                matched = "fish, ns as to type, cooked, ns as to cooking method"
            elif "steak" in x.food[i]:
                matched = "steak, ns as to type of meat, cooked, ns as to fat eaten"
            elif "beef" in x.food[i]:
                matched = "beef, ns as to cut, cooked, ns as to fat eaten"
            elif "egg" in x.food[i]:
                if "noodle" in x.food[i]:
                    matched = "beef and egg noodles, baby food, ns as to strained or junior"
                elif "plant" in x.food[i]:
                    matched = "eggplant, raw"
                else:
                    matched = "egg, whole, raw"
            elif "flour" in x.food[i]:
                matched = "bread, white"
        
        portion_info = grams.loc[(grams["Main_Food_Description"] == matched) & (grams["Portion_Description"] == unit)]["Portion_Weight_(g)"]
        if len(portion_info) > 0:
            res_dict["portion_weight"][i] = portion_info.values[0]
            _total_matched += 1
        _total_processed += 1
    return res_dict, _total_matched, _total_processed

In [22]:
pool = Pool(cpu_count())

results = []

total_matched = 0
total_processed = 0

# for row in tqdm(ingr.iterrows()):
#     results += get_weights(row)

for result in tqdm(pool.imap_unordered(get_weights, ingr.iterrows()), total=len(ingr)):
    results.append(result[0])
    total_matched += result[1]
    total_processed += result[2]

100%|██████████| 14312/14312 [02:43<00:00, 87.66it/s] 


In [23]:
final_df = pd.DataFrame(results)

In [24]:
final_df

,title,food,quantity,unit,portion_weight
0,Cocoa Syrup,"[water, sugar, cocoa, vanilla extract, kosher ...","[1 1/2, 3.0, 1 1/2, 1.0, 1/4, 2.0]","[cups, cups, cups, tablespoon, teaspoon, table...","[None, None, None, None, None, None]"
1,Zucchini Carpaccio,"[zucchini, pepper, olive oil, lemon, leek, par...","[2.0, None, 1.0, 1/2, 1.0, 1/4, None]","[None, None, tablespoon, None, None, piece, None]","[None, None, 14.0, None, None, None, None]"
2,Beet Salad,"[beets, wine vinegar, dijon mustard, caraway s...","[1 1/2, 3.0, 1.0, 1.0, 1/3, None, 6.0, 3.0, No...","[oz, tablespoons, teaspoon, teaspoon, cup, Non...","[None, None, None, None, 224.0, None, None, No..."
3,Tequila Bars,"[vanilla wafers, pine nuts, butter, tequila, l...","[12.0, 1/2, 1 1/2, 1/3, 1/2, 2.0, 1 14, 1.0]","[box, cup, sticks, cup, cup, None, can, tables...","[None, 135.0, None, None, None, None, None, None]"
4,Pasta with Pancetta and Lentils,"[kosher salt, spaghetti, olive oil, pancetta, ...","[None, 12.0, 2.0, about 2, 1/2, 4.0, 1/2, 14-o...","[None, ounces, tablespoons, ounces, cup, clove...","[None, None, None, None, None, None, None, Non..."
...,...,...,...,...,...
14307,Poblano Stuffed Cornbread,"[butter, cornmeal, all-purpose flour, baking p...","[2.0, 1.0, 1/2, 1.0, 1.0, 1/2, 1.0, 1/2, 1.0, ...","[tablespoons, cup, cup, tablespoon, teaspoon, ...","[None, None, 40.0, None, None, None, None, Non..."
14308,Spanish Style Chicken and Dumplings,"[chicken stock, saffron, olive oil, chorizo, c...","[1.0, 2.0, 2.0, 3/4, 1, 3/4, 1.0, 2 to 3, 1.0,...","[quart, pinches, tablespoons, oz, oz, oz, None...","[None, None, None, None, None, None, None, Non..."
14309,Summer Corn Salad,"[corn, belgian endive, olive oil, pepper, cila...","[4.0, 2.0, 2.0, None, 3.0, 1/2, 1/4, 1/2, 1/2,...","[ears, heads, tablespoons, None, bunch, None, ...","[None, None, None, None, None, None, None, Non..."
14310,Zucchini Stuffed Tomatoes,"[plum tomatoes, sugar, zucchini, salt, shallot...","[4.0, None, 1 1/2, 1.0, 3.0, 2.0, 3.0, 1 1/2, ...","[None, None, oz, teaspoon, None, tablespoons, ...","[None, None, None, None, None, None, None, Non..."


In [28]:
final_df.to_csv("../data/grammage.csv")

In [25]:
print ("Total matched:", total_matched)
print ("Total processed:", total_processed)

Total matched: 5891
Total processed: 132745


In [27]:
print ("Percent matched:", total_matched / total_processed *100)

Percent matched: 4.437831933406155
